In [1]:
# importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import sys
sys.path.append('..')
from Scripts.utils import (train_model_with_grid_search, load_and_predict, run_boruta)

# show all columns 
pd.set_option('display.max_columns', None)
# show entire rows
pd.set_option('display.max_rows', None)
# show all values in a cell
pd.set_option('display.max_colwidth', None)

In [2]:
# importing the datasets
clean_data_sp = pd.read_excel('../Data/clean_data_sp_w_revenue.xlsx')
clean_data_rj = pd.read_excel('../Data/clean_data_rj.xlsx')

In [3]:
# classification with random forest
from sklearn.ensemble import RandomForestClassifier

X = clean_data_rj.drop(columns=['potencial','nome'], axis=1)
y = clean_data_rj['potencial']

# Definir o modelo
model = RandomForestClassifier()

param_grid = {
    'model__n_estimators': [100, 200, 300, 400, 500],
    'model__max_depth': [None, 3, 5, 7, 10],
    'model__min_samples_split': [2, 3, 5, 10]
}

# Chamar a função para treinar o modelo com a busca em grade e salvar
train_model_with_grid_search(X, y, model, param_grid, test_size=0.2, random_state=42, save_path='../Models/rf_model_potential_rj.joblib')

Melhores parâmetros: {'model__max_depth': 5, 'model__min_samples_split': 2, 'model__n_estimators': 100}
Modelo treinado e salvo com sucesso!
Score no conjunto de teste: 0.8387096774193549


In [5]:
# apply boruta
# Criar o objeto Boruta
boruta_selector = run_boruta(X, y, model, max_iter=1000, random_state=42)

# Visualizar os resultados
boruta_results = pd.DataFrame({'feature': X.columns, 'rank': boruta_selector.ranking_})
boruta_results.sort_values(by='rank')

,feature,rank
0,população,1
30,propDomiciliosC,1
29,propDomiciliosB,1
28,propDomiciliosA,1
27,rendaMediaPop,1
26,rendaTotal,1
25,propDomiciliosIdosos,1
24,propDomiciliosCriancas,1
23,domicilios,1
21,domiciliosB,1


In [11]:
X_filtered = boruta_selector.transform(X.values)
X_sp_filtered = boruta_selector.transform(clean_data_sp.drop(columns=['nome'], axis=1).values)

In [8]:
param_grid = {
    'model__n_estimators': [100, 200, 300, 400, 500],
    'model__max_depth': [None, 1, 2, 3, 5],
    'model__min_samples_split': [2, 3, 4, 5]
}
train_model_with_grid_search(X_filtered, y, model, param_grid, test_size=0.2, random_state=42, save_path='../Models/rf_model_potential_rj.joblib')

Melhores parâmetros: {'model__max_depth': None, 'model__min_samples_split': 3, 'model__n_estimators': 300}
Modelo treinado e salvo com sucesso!
Score no conjunto de teste: 0.8709677419354839


In [14]:
# adicione uma coluna potencial_pred no clean_data_rj
clean_data_sp['potencial'] = load_and_predict(X_sp_filtered, '../Models/rf_model_potential_rj.joblib')
clean_data_rj['potencial_pred'] = load_and_predict(X_filtered, '../Models/rf_model_potential_rj.joblib')

In [16]:
# export the data
clean_data_sp.to_excel('../Data/clean_data_sp_w_revenue_w_potential.xlsx', index=False)